In [4]:
#1. 데이터셋 불러오기 
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

titanic = sns.load_dataset('titanic')

In [5]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [6]:
#2. feature 분석
titanic.describe()
#count -> 결측치(NaN)가 제거된 데이터의 수
#mean -> 평균값을 의미합니다.
#std -> 데이터가 평균으로부터 분산이 크고 작음을 보여주는 표준편차입니다.
#min -> 데이터에서 가장 작은 수치인 최소값을 나타냅니다.
#25% -> 데이터를 정렬했을 떄,하위 25%의 값을 나타냅니다.
#50% -> 데이터를 정렬했을 떄, 중앙값, 즉 데이터의 중앙을 나타냅니다.
#75% -> 데이터를 정렬했을 떄, 상위 25% 값을 나타냅니다.
#max -> 데이터에서 가장 큰 값을 나타냅니다.

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
#isnull() 함수와 sum()  함수를 이용해 각 열의 결측치 갯수를 확인
titanic.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [8]:
#3.feature engineering
#수치형 데이터의 중앙값을 계산 혹은 반환해주는 .median()사용
#계산된 중앙값을 fillna를 사용 대체, inplace = True로 변경된 값 바로 저장
titanic_age = titanic['age'].median()
titanic['age'].fillna(titanic_age, inplace = True)
#최빈값을 계산 혹은 반환해주는 .mode 사용.(리스트로 반환하기 때문에 첫번째 값인 [0]을 사용)
#계산된 최빈값을 fillna를 사용해 대체, inplace = True로 변경된 값 바로 저장
titanic_embarked = titanic['embarked'].mode()[0]
titanic['embarked'].fillna(titanic_embarked, inplace = True)

C:\Users\김준기\AppData\Local\Temp\ipykernel_28688\1924822021.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic['age'].fillna(titanic_age, inplace = True)
C:\Users\김준기\AppData\Local\Temp\ipykernel_28688\1924822021.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

In [9]:
#대체 결과를 isnull() 함수와 sum()  함수를 이용해서 확인
titanic['age'].isnull().sum()

0

In [10]:
#대체 결과를 isnull() 함수와 sum()  함수를 이용해서 확인
titanic['embarked'].isnull().sum()

0

In [11]:
#삼항 연산자(ternary operator) 구조에 기인하여, 값1 if 조건 else 값2으로 lambda 함수 생성 및 값 조정
titanic['sex'] = titanic['sex'].apply(lambda x : 0 if x == "male" else 1)
titanic['alive'] = titanic['alive'].apply(lambda x: 1 if x == 'yes' else 0)
titanic['embarked'] = titanic['embarked'].apply(lambda x:0 if x == 'C' else (1 if x == 'Q' else 2))

titanic['sex'].head()

0    0
1    1
2    1
3    1
4    0
Name: sex, dtype: int64

In [12]:
#변환 결과를 head 함수를 이용해 확인
titanic['alive'].head()

0    0
1    1
2    1
3    1
4    0
Name: alive, dtype: int64

In [13]:
#변환 결과를 head 함수를 이용해 확인
titanic['embarked'].head()

0    2
1    0
2    2
3    2
4    2
Name: embarked, dtype: int64

In [14]:
#변환 결과를 head 함수를 이용해 확인
titanic['Family Size'] = titanic['sibsp'] + titanic['parch'] + 1
titanic['Family Size'].head()

0    2
1    2
2    1
3    2
4    1
Name: Family Size, dtype: int64

In [15]:
#4.모델 학습시키기
titanic = titanic[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'Family Size']]
X = titanic.drop('survived', axis = 1)
y = titanic['survived']

In [16]:
#심심해서 해본 accuracy 제일 높은거 고르기
#Logistic Regression

best_accuracy = 0
best_state = 0
for x in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = x)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    logi = LogisticRegression()
    logi.fit(X_train, y_train)

    y_answer = logi.predict(X_test)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    logi = LogisticRegression()
    logi.fit(X_train, y_train)

    y_answer = logi.predict(X_test)
    accuracy = accuracy_score(y_test, y_answer)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_state = x
print(f'Best accuracy: {best_accuracy}, Best state: {best_state}')
print(f'Classification Report: \n{classification_report(y_test, y_answer)}')

Best accuracy: 0.8603351955307262, Best state: 18
Classification Report: 
              precision    recall  f1-score   support

           0       0.79      0.82      0.81       113
           1       0.67      0.62      0.65        66

    accuracy                           0.75       179
   macro avg       0.73      0.72      0.73       179
weighted avg       0.75      0.75      0.75       179



In [17]:
#그냥 넣고싶었던 것
mse = mean_squared_error(y_test, y_answer)
r2 = r2_score(y_test, y_answer)
print(f'Mean Squared Error: {mse}')
print(f'r2score: {r2}')

Mean Squared Error: 0.25139664804469275
r2score: -0.08004827031375683


In [18]:
#이번에는 다르게 줘봤습니다. for을 하나 더 써서 최적의 test size와 그에 상응하는 random state를 찾아 최고의 정확도를  찾아내는 것이 목표입니다.
#Random Forest
best_accuracy = 0
best_state = 0
best_test_size = 0
#np.arange(start, stop, step)
for test_size in np.arange(0.1, 1, 0.05):
    for x in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = x)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        tree = DecisionTreeClassifier(random_state = x)
        tree.fit(X_train, y_train)

        y_answer = tree.predict(X_test)
        accuracy = accuracy_score(y_test, y_answer)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_test_size = test_size
            best_state = x
print(f'best accuracy: {best_accuracy}')
print(f'best test size: {best_test_size}')
print(f'best state: {best_state}')

best accuracy: 0.8777777777777778
best test size: 0.1
best state: 72


In [19]:
#best_mse= 1000000
#best_state = 0
#best_test_size = 0
#n_estimators = [100, 200, 500]
#learning_rate = [0.01, 0.1, 0.2, 0.3]
#max_depth = [3, 6, 10, 15]
#best_n_estimators = 0
#best_learning_rate = 0
#best_max_depth = 0
#for test_size in np.arange(0.1, 0.6, 0.05):
    #for state in range(50):
        #X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = test_size, random_state = state)
        #scaler = StandardScaler()
        #X_train = scaler.fit_transform(X_train)
        #X_test = scaler.transform(X_test)

        #for esti in n_estimators:
            #for rate in learning_rate:
                #for depth in max_depth:
                    #xgboost = xgb.XGBRegressor(n_estimators = esti, learning_rate = rate, max_depth = depth, random_state = state)
                    #xgboost.fit(X_train, y_train)
                    #y_answer = xgboost.predict(X_test)
                    
                    #mse = mean_squared_error(y_test, y_answer)
                    #if mse < best_mse:
                        #best_mse = mse
                        #best_state = state
                        #best_test_size = test_size
                        #best_n_estimators = esti
                        #best_learning_rate = rate
                        #best_max_depth = depth

#print(f'best mse: {best_mse}')
#print(f'best test size: {best_test_size}')
#print(f'best state: {best_state}')
#print(f'best n estimators: {best_n_estimators}')
#print(f'best learning rate: {best_learning_rate}')
#print(f'best max depth: {best_max_depth}')
#그....말도 안되게 오래걸려서 패스

In [22]:
#XGBoost
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

xgboost = xgb.XGBRegressor(n_estimators = 100, learning_rate = 0.1, max_depth = 3, random_state = 42)
xgboost.fit(X_train, y_train)
y_answer = xgboost.predict(X_test)
mse = mean_squared_error(y_test, y_answer)
print(f'모델의 MSE: {mse}')

모델의 MSE: 0.12981004899201257
